In [31]:
import pandas as pd
import re
import string

# Load your data
df = pd.read_csv('ulasan_femaledaily.csv')

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned'] = df['Reviews'].astype(str).apply(preprocess_text)

# Example lexicon (expand as needed)
positive_words = set([
    
    'bagus', 'suka', 'mantap', 'enak', 'puas', 'recommended', 'cocok', 'baik', 'top',
    'nyaman','senang', 'gembira', 'bahagia', 'ceria', 'riang', 'sukacita', 'bangga', 'puas',
    'cinta', 'sayang', 'kasih', "rindu", "manis", "romantis", "harmonis", "mesra",
    "tenang", "damai", "nyaman", "tentram", "adem", "sejuk", "kebahagiaan", "ketenangan",
    "rajin", "giat", "semangat", "aktif", "produktif", "bersemangat", "antusias", "energik",
    "menarik", "seru", "mengagumkan", "memukau", "luar biasa", "fantastis", "menyenangkan", "asyik",
    "sehat", "bugar", "segar", "kuat", "fit", "vit", "berstamina", "prima",
    "sukses", "berhasil", "unggul", "juara", "menang", "prestasi", "kemenangan", "kesuksesan",
    "baik", "bagus", "positif", "mulia", "terpuji", "bermanfaat", "berkualitas", "sempurna",
    "mudah", "lancar", "sederhana", "praktis", "efisien", "cepat", "mudah", "tanpa hambatan",
    "harapan", "optimis", "percaya", "yakin", "keyakinan", "mimpi", "ambisi", "keinginan",
    "indah", "cantik", "elok", "menawan", "memesona", "molek", "rupawan", "anggun",
    "cerdas", "pintar", "brilian", "genius", "berbakat", "pandai", "kreatif", "inovatif",
    "jujur", "amanah", "terpercaya", "tulus", "ikhlas", "sungguh-sungguh", "dedikasi", "integritas",
    "kaya", "makmur", "sejahtera", "berkelimpahan", "beruntung", "berkecukupan", "sukses finansial",
    "sabar", "tabah", "tahan banting", "ulet", "tekun", "gigih", "pantang menyerah", "ketekunan",
    "sopan", "ramah", "baik hati", "murah senyum", "bersahabat", "hangat", "menghormati", "simpatik",
    
    # Emosi positif
    "sumringah", "berbunga-bunga", "senang bukan main", "girang", "bahagia",
    "adem ayem", "nyaman banget", "tentram", "lega", "plong",
    "bersemangat 45", "on fire", "greget", "spirit", "antusias",
    
    # Sifat positif
    "murah senyum", "ringan tangan", "empati", "tulus ikhlas", "baik hati",
    "akrab", "cair", "easygoing", "santuy", "kalem",
    "cemerlang", "brilian", "visioner", "kreatif", "inovatif",
    
    # Situasi positif
    "jaya", "gemilang", "mentereng", "moncer", "top markotop",
    "memukau", "mengesankan", "anggun", "elok", "cantik jelita",
    "gampang", "simpel", "nggak ribet", "lancar jaya", "mulus",
    
    # Slang/gaul positif
    "mantap", "juara", "keren abis", "oke banget", "wow",
    "gemoy", "lucu banget", "imut", "charming", "menggemaskan",
    "nyaman banget", "ngelegin", "bikin adem", "vibesnya enak", "cozy",
    "luv", "soulmate", "jodoh", "chemistry", "kompak",
    "produktif", "hustle", "growth", "upgrade", "glowing"
    
    ])

negative_words = set([
    
    'jelek', 'buruk', 'tidak', 'mengecewakan', 'parah', 'mahal', 'kurang', 'gagal',
    'benci', 'tidak suka',"marah", "kesal", "geram", "jengkel", "dengkik", "murka", "ngamuk", "sakit hati",
    "sedih", "duka", "nestapa", "pilu", "sengsara", "terluka", "merana", "putus asa",
    "takut", "cemas", "khawatir", "was-was", "gelisah", "parno", "fobia", "panik",
    "malas", "lamban", "ogah", "enggan", "lemas", "lesu", "tak bergairah", "mogok",
    "bosan", "jenuh", "monoton", "membosankan", "hambar", "tanpa arti", "tdk menarik",
    "sakit", "nyeri", "perih", "pedih", "luka", "tersiksa", "menderita", "kram",
    "gagal", "kalah", "bangkrut", "kolaps", "rugi", "tidak lulus", "tersandung",
    "benci", "dendam", "muak", "antipati", "jijik", "alergi", "menyebalkan", "menjengkelkan",
    "buruk", "jelek", "rendah", "hancur", "rusak", "berantakan", "kacau", "mengerikan",
    "susah", "sulit", "rumit", "berat", "ribet", "merepotkan", "nyusahin", "menyulitkan",
    "bodoh", "tolol", "dungu", "bebal", "bloon", "otak udang", "tdk kompeten",
    "menipu", "bohong", "kecurangan", "licik", "curang", "penipuan", "kadaluarsa",
    "kotor", "jorok", "kumal", "dekil", "jijik", "kotoran", "polusi", "tercemar",
    "terlambat", "telat", "molor", "tertunda", "keburu", "kehabisan", "keduluan",
    "penakut", "lemah", "pengecut", "takut-takutan", "ciut", "gugup", "gemetar",
    "minder", "rendah diri", "pesimis", "putus ada", "menyerah", "down", "depresi",
    "kesal", "jengkel", "sebel", "gemas", "geram", "dongkol", "ngambek", 
    "sakit hati", "kecewa berat", "galau", "hati hancur", "merana", "nestapa",
    "deg-degan", "waswas", "ciut hati", "gugup", "grogi", "parno", "tremor",
    "jenuh", "suntuk", "garing", "hambar", "ngebosenin", "mageran",
    
    # Sifat negatif
    "minder", "rendah diri", "malu-maluin", "pesimis", "putus asa",
    "molor", "rebahan", "ogah-ogahan", "leha-leha", "teler",
    "pelit", "matre", "sok penting", "egois", "maunya sendiri",
    "ceplas-ceplos", "bawel", "judes", "nyinyir", "kasar",
    
    # Situasi negatif
    "bangkrut", "kandas", "mentok", "jatuh bangun", "gagal total",
    "reyot", "kumal", "belel", "dekil", "berantakan",
    "ribet", "ruwet", "mumet", "pusing tujuh keliling", "nyusahin",
    
    # Slang/gaul negatif
    "bete", "gemesin", "nyesek", "gregetan", "kece berat",
    "ngenes", "lebay", "alay", "norak", "kampungan",
    "badmood", "badai", "ambyar", "bucin sakit", "ghosting",
    "baper", "troll", "toxic", "drama queen", "oversensitive"
    
    ])

def lexicon_sentiment(text):
    words = text.split()
    pos = sum(1 for w in words if w in positive_words)
    neg = sum(1 for w in words if w in negative_words)
    if pos > neg:
        return 'positive'
    elif neg > pos:
        return 'negative'
    else:
        return 'neutral'

df['lexicon_sentiment'] = df['cleaned'].apply(lexicon_sentiment)
print(df[['Reviews', 'lexicon_sentiment']].head())

                                             Reviews lexicon_sentiment
0  Awal pake skincare aku pake natural republic, ...           neutral
1  aku pake ini dari aku kuliah sampe sekarang ka...          positive
2  Tekstur nya gel jadi ringan di kulit terus mel...          positive
3  Produk serba bisa, buat rambut bisa buat pelem...          positive
4  Suka banget pake produk ini, melembabkan sekal...          positive


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# If you have a 'Label' column, use it. Otherwise, use 'lexicon_sentiment' as pseudo-labels
X = df['cleaned']
y = df['lexicon_sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 1. Predict sentiment for all reviews using the trained model
all_reviews_vec = vectorizer.transform(df['cleaned'])
predicted_labels = model.predict(all_reviews_vec)

# 2. Add the predicted labels to the DataFrame
df['nb_sentiment'] = predicted_labels

# 3. If your model outputs English labels, map them to Indonesian
label_mapping = {
    'positive': 'positif',
    'negative': 'negatif',
    'neutral': 'neutral'
}
df['nb_sentiment'] = df['nb_sentiment'].map(label_mapping).fillna(df['nb_sentiment'])

# 4. Save or display the DataFrame
print(df[['Reviews', 'nb_sentiment']].head())
# Optionally save to CSV
# df.to_csv('ulasan_femaledaily_with_nb_sentiment.csv', index=False)

Accuracy: 0.7718120805369127

Classification Report:
               precision    recall  f1-score   support

    negative       0.00      0.00      0.00        17
     neutral       0.00      0.00      0.00        51
    positive       0.77      1.00      0.87       230

    accuracy                           0.77       298
   macro avg       0.26      0.33      0.29       298
weighted avg       0.60      0.77      0.67       298

                                             Reviews nb_sentiment
0  Awal pake skincare aku pake natural republic, ...      positif
1  aku pake ini dari aku kuliah sampe sekarang ka...      positif
2  Tekstur nya gel jadi ringan di kulit terus mel...      positif
3  Produk serba bisa, buat rambut bisa buat pelem...      positif
4  Suka banget pake produk ini, melembabkan sekal...      positif


c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [33]:
# Create a new DataFrame with only the desired columns
result_df = df[['Reviews', 'nb_sentiment', 'lexicon_sentiment']]

# Display the new DataFrame
print(result_df.head())

# Optionally, save to CSV
result_df.to_csv('sentiment_results.csv', index=False)

                                             Reviews nb_sentiment  \
0  Awal pake skincare aku pake natural republic, ...      positif   
1  aku pake ini dari aku kuliah sampe sekarang ka...      positif   
2  Tekstur nya gel jadi ringan di kulit terus mel...      positif   
3  Produk serba bisa, buat rambut bisa buat pelem...      positif   
4  Suka banget pake produk ini, melembabkan sekal...      positif   

  lexicon_sentiment  
0           neutral  
1          positive  
2          positive  
3          positive  
4          positive  
